In [1]:
from unet import *
import functions
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import trange
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
import scipy.io
# tensor summary import
from torchsummary import summary

/home/charles/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = "checkpoints_private/2023_03_30.pt"
Net = build_unet()
Net.load_state_dict(torch.load(checkpoint))
Net.eval()
# check if CUDA is available, and set it as the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("==> Device: {}".format(device))

# move model to the device
Net.to(device)

# define loss function
#loss_function = nn.MSELoss()
loss_function = nn.L1Loss()

####### HYPERPARAMETERS #######

# patch size
patch_size = 512

# batch size
batch_size = 1

# keep track of the best validation loss
valid_loss_min = np.Inf

##### LOSS  #######

# keep track of training and validation loss
test_loss = []
valid_loss = []

# initialize the early_stopping object
# early_stopping = EarlyStopping(patience=early_stopping, verbose=True)


==> Device: cuda


In [3]:
# read the list.txt as a space separated dataframe
df = pd.read_csv('Sony_test_list.csv', sep=',', header=None)
#df = pd.read_csv('list_test.csv', sep=',', header=None)

# split df into input, label columns
input_df = df.iloc[:, 0]
label_df = df.iloc[:, 1]

# create a list of tuples
image_label_list = list(zip(input_df, label_df))

# create a dataset object
dataset = Dataset()

# create a dataloader object
dataloader = DataLoader(image_label_list, batch_size=batch_size, shuffle=False)

print('number of images in dataset: {}'.format(len(dataloader)))

result_dir = "results_test/"
if not os.path.isdir(result_dir):
                os.makedirs(result_dir)

number of images in dataset: 598


In [12]:
epoch_loss = 0
cnt = 0

t = trange(len(dataloader), desc='Loss: ', leave=True)
for epoch in t:

    Net.eval()

    for input, label in dataloader:
        # load the first image and label
        print(' ------------ new batch --------------')
        print("Input: " + str(input))
        print("Label: " + str(label))
        input = input[0]
        label = label[0]

        exposure_input = float(((input.split('_'))[-1]).rstrip('s.ARW'))
        exposure_label = float(((label.split('_'))[-1]).rstrip('s.ARW'))
        ratio = min(exposure_label / exposure_label, 300)

        # image = rawpy.imread(input)
        image = functions.pack_raw_sony(rawpy.imread(input)) * ratio
        # image = np.expand_dims(image, axis=0)
        label = rawpy.imread(label)

        # convert the image and label to numpy arrays
        # image = image.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
        label = label.postprocess(
            use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)

        # convert the image and label to float32 data type
        # image = np.float32(image / 65535.0)
        label = np.float32(label / 65535.0)
        # label = np.expand_dims(label, axis=0)

        ####### POSTPROCESSING #######

        # crop the image and label to 512 x 512
        image, label = functions.random_crop(image, label, patch_size)


        # convert the image and label to tensors
        image = torch.from_numpy(np.expand_dims(
            np.transpose(image, (2, 0, 1)), axis=0))
        label = torch.from_numpy(np.expand_dims(
            np.transpose(label, (2, 0, 1)), axis=0))

        # move the image and label to the device
        image, label = image.to(device), label.to(device)

        # forward pass
        output = Net(image)

        # calculate the loss
        loss = loss_function(output, label)

        if cnt % 100 == 0:
            label_image = label[0].cpu().detach().numpy().transpose(1, 2, 0)

            output_image = output[0].cpu().detach().numpy().transpose(1, 2, 0)
            output_image = np.minimum(np.maximum(output_image, 0), 1)
            output_image = np.concatenate((label_image, output_image), axis=1)

            if not os.path.isdir(result_dir + '%04d' % epoch):
                os.makedirs(result_dir + '%04d' % epoch)
            functions.toimage(output_image * 255, high=255, low=0, cmin=0, cmax=255).save(
                result_dir + '%04d/%05d_' % (epoch, ratio) + str(os.path.basename(input)[:-3]) + 'jpg')

        # append loss to the list
        test_loss.append(loss.item())

        cnt = cnt + 1


# plot loss
plt.plot(test_loss)
plt.title("Test Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

# plot


Loss:   0%|          | 0/598 [00:00<?, ?it/s]

 ------------ new batch --------------
Input: ('images/SonyImages/Sony/short/10003_00_0.04s.ARW',)
Label: ('images/SonyImages/Sony/long/10003_00_10s.ARW',)
 ------------ new batch --------------
Input: ('images/SonyImages/Sony/short/10003_00_0.1s.ARW',)
Label: ('images/SonyImages/Sony/long/10003_00_10s.ARW',)
 ------------ new batch --------------
Input: ('images/SonyImages/Sony/short/10003_01_0.04s.ARW',)
Label: ('images/SonyImages/Sony/long/10003_00_10s.ARW',)
 ------------ new batch --------------
Input: ('images/SonyImages/Sony/short/10003_01_0.1s.ARW',)
Label: ('images/SonyImages/Sony/long/10003_00_10s.ARW',)
 ------------ new batch --------------
Input: ('images/SonyImages/Sony/short/10003_02_0.04s.ARW',)
Label: ('images/SonyImages/Sony/long/10003_00_10s.ARW',)
 ------------ new batch --------------
Input: ('images/SonyImages/Sony/short/10003_02_0.1s.ARW',)
Label: ('images/SonyImages/Sony/long/10003_00_10s.ARW',)
 ------------ new batch --------------
Input: ('images/SonyImages/S